# Master Discogs Database

In [ ]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))


################################################################################
## General Stuff
################################################################################
from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote
from collections import Counter
from searchUtils import findExt, findSubExt, findPatternExt, findNearest
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from listUtils import getFlatList
from time import sleep


################################################################################
## Music Stuff
################################################################################

### MultiArtist
from multiArtist import multiartist

### My Music DB
from myMusicDBMap import myMusicDBMap
from musicDBMap import musicDBMap
from matchDBArtist import matchDBArtist
from masterDBMatchClass import masterDBMatchClass

### Master DB code
from masterdb import masterdb
from mainDB import mainDB


import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

# Create Full DB

In [ ]:
### Master DB code
%load_ext autoreload
%autoreload
from masterdb import masterdb
from mainDB import mainDB

start, cmt = clock("Creating DB")
maindb = mainDB(mdb=None, create=True, debug=True)
print("\n\n{0}\n".format("Full DB"))
maindb.setDBFull("AllMusic")
#maindb.setDBFull("MusicBrainz")
print("\n\n{0}\n".format("Known DB"))
elapsed(start, cmt)

# Main DB

In [ ]:
### Master DB code
%load_ext autoreload
%autoreload
from masterdb import masterdb
from mainDB import mainDB

In [ ]:
maindb = mainDB(mdb=None, create=False, debug=False)
maindb.loadDBDataMap()

In [ ]:
mdbmap = musicDBMap("Music", init=False)

In [ ]:
myDF = mdbmap.getDF().T

In [ ]:
from pandas import DataFrame
tmp = myDF[~myDF["AllMusic"].isna()]
mbToGet = DataFrame(tmp[tmp["MusicBrainz"].isna()]["AllMusic"]).copy(deep=True)

In [ ]:
mbToGet

In [ ]:
matchData  =  mdbmc.getDBMatchData("AllMusic")

In [ ]:
toMatch = {"Music": []}
for idx,row in mbToGet.iterrows():
    amID = row["AllMusic"]
    primKey = mdbmaps["AllMusic"].getPrimaryKeyFromID(amID)
    #print(idx,'\t',primKey)
    try:
        albums = matchData[primKey]
    except:
        continue
    toMatch["Music"].append([idx[0], idx[1], albums])

In [ ]:
mdbmaps["MusicBrainz"] = musicDBMap("MusicBrainz", init=False)


In [ ]:
loadAll = True
if loadAll:
    mdbmaps = {}
    for db in maindb.getDBs():
        print("{0: <30}".format(db), end="")
        mdbmaps[db] = musicDBMap(db, init=False)
    #mdbmaps = {db: musicDBMap(db, init=False) for db in maindb.getDBs()}
else:
    mdbmaps = {}
    #mdbmaps["AllMusic"] = musicDBMap("AllMusic", init=False)
    mdbmaps["MusicBrainz"] = musicDBMap("MusicBrainz", init=False)
    if False:
        mdbmaps["AllMusic"] = musicDBMap("AllMusic", init=False)
        mdbmaps["MusicBrainz"] = musicDBMap("MusicBrainz", init=False)
        mdbmaps["Discogs"] = musicDBMap("Discogs", init=False)
        mdbmaps["LastFM"] = musicDBMap("LastFM", init=False)
        mdbmaps["RockCorner"] = musicDBMap("RockCorner", init=False)
        mdbmaps["AceBootlegs"] = musicDBMap("AceBootlegs", init=False)
        mdbmaps["MusicStack"] = musicDBMap("MusicStack", init=False)
        mdbmaps["CDandLP"] = musicDBMap("CDandLP", init=False)

In [ ]:
results = {}
for db,togets in toMatch.items():
    for toget in togets:
        artistName = toget[0]
        artistID   = toget[1]
        albums     = toget[2]
        mdbMatcher = matchDBArtist(maindb)
        mdbMatcher.setArtistInfo(artistName, artistID, albums)
        #mdbMatcher.findPotentialArtistNameMatches()
        mc = mdbMatcher.findPotentialArtistAlbumMatchesByDB('MusicBrainz')
        results[(artistName,artistID)] = mc
        print(artistName,artistID)
        #mcs = mdbMatcher.findPotentialArtistAlbumMatches()

In [ ]:
for primKey,match in results.items():
    mdbmap.addArtistData(primKey[0], primKey[1], "MusicBrainz", match.matchID)

In [ ]:
mdbmap.save()

****

# Primary DB Matching

In [ ]:
mdbmc = masterDBMatchClass(maindb, mdbmaps)

In [ ]:
    mdbMatcher = matchDBArtist(maindb)
    mdbMatcher.setArtistInfo(artistName, artistID, ["Bad Blood"])
    #mdbMatcher.findPotentialArtistNameMatches()
    mc = mdbMatcher.findPotentialArtistAlbumMatchesByDB('AllMusic')
    mcs = mdbMatcher.findPotentialArtistAlbumMatches()

In [ ]:
mdbMatcher = matchDBArtist(maindb)
mdbMatcher.setArtistInfo(artistName, artistID, artistAlbums)
    mcs    = mdbMatcher.findPotentialArtistAlbumMatches()
    retval = [artistName,artistID,mcs]


In [ ]:
if False:
    mdf = mdbmaps["MusicBrainz"].getDF()
    saveFile(idata=list(mdf.T[mdf.T["MusicBrainz"].isna()].index), ifile="togetMB.p")

In [ ]:
if False:
    toget   = getFile(ifile="togetMB.p")
    #dbm = mdbmc.getDBMatchData("MusicBrainz")
    print(len(toget))

In [ ]:
if False:
    toMatchs = {}
    i = 0
    toMatchs[i] = {"MusicBrainz": []}
    for idx in toget:
        val = dbm.get(idx)
        if val is not None:
            toMatchs[i]["MusicBrainz"].append([idx[0],idx[1],albums])
            if len(toMatchs[i]["MusicBrainz"]) >= 5000:
                i+= 1
                toMatchs[i] = {"MusicBrainz": []}

In [ ]:
if False:
    for primaryKey,albums in dbm.items():
        idx = primaryKey[1]
        if idx in toget:
            toMatch["MusicBrainz"].append([primaryKey[0],primaryKey[1],albums])

In [ ]:
#toMatch   = mdbmc.getDataToMatch("MusicBrainz", maxValues=1000, maxAlbums=50)
#toMatch

In [ ]:
if False:
    mdbmaps["AllMusic"].addArtist("Herbert von Karajan", "0000031181")
    mdbmaps["AllMusic"].addArtistData("Herbert von Karajan", "0000031181", "AllMusic", "0000031181")

    mdbmaps["AllMusic"].addArtist("Lata Mangeshkar", "0000132940")
    mdbmaps["AllMusic"].addArtistData("Lata Mangeshkar", "0000132940", "AllMusic", "0000132940")

    mdbmaps["AllMusic"].save()

# Matching Code

In [ ]:
from tqdm import tqdm
from multiprocessing import Pool
from functools import partial
import time

def matchDBArtistWithAlbums(item, *args, **kwargs):    
    #time.sleep(0.0025)

    artistName   = item[0]
    artistID     = item[1]
    artistAlbums = item[2]
    
    mdbMatcher = matchDBArtist(maindb)
    mdbMatcher.setArtistInfo(artistName, artistID, artistAlbums)
    mcs    = mdbMatcher.findPotentialArtistAlbumMatches()
    retval = [artistName,artistID,mcs]
    return retval
    #result = findNearest(name, artists, 1, kwargs['cutoff'])


def multiProc(func, argument_list, num_processes):
    pool = Pool(processes=num_processes)
    result_list_tqdm = []
    for result in tqdm(pool.imap(func=func, iterable=argument_list), total=len(argument_list)):
        result_list_tqdm.append(result)
    return result_list_tqdm



def saveMapData(mdbmap, result_list):
    for item in result_list:
        artistName = item[0]
        artistID   = item[1]
        mcs        = item[2]
        mdbmap.addArtist(artistName, artistID)
        for db,mc in mcs.items():
            matchID    = mc.matchID
            matchScore = mc.matchScore
            if matchID is not None:
                mdbmap.addArtistData(artistName, artistID, db, matchID)

    mdbmap.save()

In [ ]:
for i in range(1):
    #for db in ["AllMusic", "MusicBrainz", "LastFM", "RockCorner", "CDandLP", "RateYourMusic", "MusicStack"]:
    for db in ["Music"]: #, "MusicBrainz", "LastFM", "RockCorner", "CDandLP", "RateYourMusic", "MusicStack"]:
        mdbmc = masterDBMatchClass(maindb, mdbmaps)
        #toMatch   = mdbmc.getDataToMatch(db, maxValues=500, maxAlbums=50000)
        
        if len(toMatch) == 0:
            continue
        

        num_processes = 3
        func = matchDBArtistWithAlbums
        pfunc = partial(matchDBArtistWithAlbums, cutoff=0.95) # Giving some arguments for kwargs
        #argument_list = list(inputs.items()) # [random.randint(0, 100) for _ in range(num_jobs)]
        dbName = list(toMatch.keys())[0]
        argument_list = toMatch[dbName]
        if len(argument_list) == 0:
            break
        print("Running imap multiprocessing for {0} artists ...".format(len(argument_list)))
        result_list = multiProc(func=pfunc, argument_list=argument_list,
                                               num_processes=num_processes)


        break
        saveMapData(mdbmaps[dbName], result_list)
        sleep(10)

        #mdbmc.matchMutualMaps()

In [ ]:
result_list

In [ ]:
toMatch

In [ ]:
mbdf = mdbmaps["MusicBrainz"].getDF().T

In [ ]:
maindb.dbdata["MusicBrainz"]["Utils"].getArtistID("1660fbb6-c55e-4ad3-8342-c424162c30a6")

In [ ]:
maindb.dbdata["MusicBrainz"]["Utils"].getArtistID("c685765c-b778-4ed6-9035-8f7088431765")

In [ ]:
maindb.dbdata["MusicBrainz"]["Utils"].getArtistID("5bf05dd4-3ac1-4b8b-b9a1-163790d10192")

In [ ]:
mdbmaps["MusicBrainz"].addArtist("The Highwomen", "286831209235815715269312378569239355331")
mdbmaps["MusicBrainz"].addArtistData("The Highwomen", "286831209235815715269312378569239355331", "MusicBrainz", "286831209235815715269312378569239355331")
mdbmaps["MusicBrainz"].addArtistData("The Highwomen", "286831209235815715269312378569239355331", "AllMusic", "0003859078")
mdbmaps["MusicBrainz"].save()

In [ ]:
for idx in mbdf.index:
    if idx[0].find("Highway") != -1:
        print(idx)

In [ ]:
toget   = getFile(ifile="togetMB.p")
for primaryKey in toget:
    mdbmaps["MusicBrainz"].removeArtist(primaryKey[0], primaryKey[1])
mdbmaps["MusicBrainz"].save()

In [ ]:
if False:
    results = {}
    for item in matchData[:100]:
        print(item[0])
        mdbMatcher.setArtistInfo(artistName, artistID, artistAlbums)
        #mdbMatcher.findPotentialArtistNameMatches()
        #mc = mdbMatcher.findPotentialArtistAlbumMatchesByDB('Discogs')
        mcs = mdbMatcher.findPotentialArtistAlbumMatches()
        results[(artistName,artistID)] = {db: [mc.matchID, mc.matchScore] for db, mc in mcs.items()}

In [ ]:
df = mdbmap.getDF().T

In [ ]:
from pandas import Series
matchData  = mdbmc.getDBMatchData("AllMusic")
sortedData = Series({primaryKey: len(albums) for primaryKey,albums in matchData.items()}).sort_values(ascending=False).to_dict()

In [ ]:
sortedData

In [ ]:
matchData